In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
%matplotlib inline

In [2]:
batch_size =100


# Download Data
cifar_train = dset.CIFAR10("./",train=True,transform=transforms.ToTensor(),target_transform=None,download=True)
cifar_test= dset.CIFAR10("./",train=False,transform=transforms.ToTensor(),target_transform=None,download=True)

#mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
#mnist_test  = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

# Set Data Loader(input pipeline)

train_loader = torch.utils.data.DataLoader(dataset=cifar_train,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=cifar_test,batch_size=batch_size)

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [9]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


class UnFlatten(nn.Module):
    def forward(self, input, size=1024):
        return input.view(input.size(0), size, 1, 1)

class VAE(nn.Module):
    def __init__(self, image_channels=3, h_dim=1024, z_dim=32):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2,padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2,padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2,padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2,padding=1),
            nn.ReLU(),
            Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(h_dim, 128, kernel_size=4, stride=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=6, stride=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=6, stride=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=6, stride=2),
            nn.Sigmoid(),
        )
        
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_().cuda()
        # return torch.normal(mu, std)
        esp = torch.randn(*mu.size()).cuda()
        z = mu + std * esp
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = self.decoder(z)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar

In [10]:
# build model
vae = VAE()
if torch.cuda.is_available():
    vae.cuda()

In [11]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum') #x.view(-1, 1024)
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [12]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        if torch.cuda.is_available():
            data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            if torch.cuda.is_available():
                data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [17]:
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/50000 (0%)]	Loss: 1845.835625
Train Epoch: 1 [10000/50000 (20%)]	Loss: 1818.588281
Train Epoch: 1 [20000/50000 (40%)]	Loss: 1799.873906
Train Epoch: 1 [30000/50000 (60%)]	Loss: 1784.241563
Train Epoch: 1 [40000/50000 (80%)]	Loss: 1809.230625
====> Epoch: 1 Average loss: 1820.7228
====> Test set loss: 1826.2763
Train Epoch: 2 [0/50000 (0%)]	Loss: 1811.452656
Train Epoch: 2 [10000/50000 (20%)]	Loss: 1813.932969
Train Epoch: 2 [20000/50000 (40%)]	Loss: 1804.663281
Train Epoch: 2 [30000/50000 (60%)]	Loss: 1829.004531
Train Epoch: 2 [40000/50000 (80%)]	Loss: 1821.543437
====> Epoch: 2 Average loss: 1820.7424
====> Test set loss: 1826.4750
Train Epoch: 3 [0/50000 (0%)]	Loss: 1827.175156
Train Epoch: 3 [10000/50000 (20%)]	Loss: 1807.851875
Train Epoch: 3 [20000/50000 (40%)]	Loss: 1808.386719
Train Epoch: 3 [30000/50000 (60%)]	Loss: 1816.398438
Train Epoch: 3 [40000/50000 (80%)]	Loss: 1805.667031
====> Epoch: 3 Average loss: 1820.5755
====> Test set loss: 1826.2224
Train Epoc